# C2-C6

In [215]:
import pandas as pd
import os
import pathlib

def fill_quarts(
        data: pd.DataFrame,
        type_appareil: str,
) -> pd.DataFrame:
    if type_appareil == 'C2-C6':

        shift_data = data.shift(freq='15min')
        data = pd.concat([data, shift_data], axis=0)

    if type_appareil == 'C6-C20':
        shift_data = data.shift(freq='15min')
        data = pd.concat([data, shift_data], axis=0)
        for i in range(2):
            
            shift_data = shift_data.shift(freq='15min')

            data = pd.concat([data, shift_data], axis=0)

    return (data.sort_index())


month = '02'
year = '2022'
type_appareil = [
    'C2-C6',
    'C6-C20',
]

file_directories = [f for f in pathlib.Path().glob(f'../test_data/{year}/{month}/*{type_appareil[0]}.Asc')]

C2C6_data = pd.DataFrame()
for file in file_directories:
    asc_data = pd.read_table(file)
    asc_data['Sampling date'] = pd.to_datetime(asc_data['Sampling date']).dt.round('15min')
    asc_data.set_index('Sampling date', inplace=True)
    C2C6_data = pd.concat([C2C6_data, asc_data])

C2C6_data.sort_index(inplace=True)

C2C6_data = fill_quarts(
    data=C2C6_data,
    type_appareil=type_appareil[0]
    )
C2C6_data = C2C6_data[~C2C6_data.index.duplicated(keep='first')]

In [216]:
C2C6_data

,Volume,ETHANE,ETHYLENE,PROPANE,PROPENE,I-BUTANE,N-BUTANE,ACETYLENE,TRANS-2-BUTENE,1-BUTENE,...,3-ME-PENTANE,N-HEXANE,ISOPRENE,2-ME-1-PENTENE,2-4-DIME-PENTANE,2-3-DIME-PENTANE,2-ME-HEXANE,BENZENE,Unnamed: 32,Unnamed: 5
Sampling date,,,,,,,,,,,,,,,,,,,,,
2022-01-31 23:30:00,100.88,1.78,0.98,0.96,0.04,0.10,0.32,0.0,0.03,0.00,...,0.0,0.00,0.00,0.63,0.00,0.00,0.02,0.00,NaN,NaN
2022-01-31 23:45:00,100.88,1.78,0.98,0.96,0.04,0.10,0.32,0.0,0.03,0.00,...,0.0,0.00,0.00,0.63,0.00,0.00,0.02,0.00,NaN,NaN
2022-02-01 00:00:00,100.91,1.77,0.91,0.39,0.00,0.11,0.31,0.0,0.04,0.00,...,0.0,0.00,0.00,0.59,0.00,0.00,0.00,0.00,NaN,NaN
2022-02-01 00:15:00,100.91,1.77,0.91,0.39,0.00,0.11,0.31,0.0,0.04,0.00,...,0.0,0.00,0.00,0.59,0.00,0.00,0.00,0.00,NaN,NaN
2022-02-01 00:30:00,100.95,1.80,0.89,0.40,0.00,0.10,0.29,0.0,0.00,0.00,...,0.0,0.03,0.00,0.62,0.00,0.02,0.00,0.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-28 22:15:00,101.65,2.87,8.44,1.20,0.32,0.24,0.43,0.0,0.05,0.00,...,0.0,0.00,1.04,0.00,0.00,0.00,0.02,0.04,NaN,NaN
2022-02-28 22:30:00,101.65,2.79,8.54,1.19,0.46,0.19,0.39,0.0,0.19,0.18,...,0.0,0.00,1.13,0.08,0.00,0.02,0.00,0.07,NaN,NaN
2022-02-28 22:45:00,101.65,2.79,8.54,1.19,0.46,0.19,0.39,0.0,0.19,0.18,...,0.0,0.00,1.13,0.08,0.00,0.02,0.00,0.07,NaN,NaN


# C6-C20

In [217]:
import pandas as pd
import os
import pathlib

type_appareil = [
    'C2-C6',
    'C6-C20',
]

file_directories = [f for f in pathlib.Path().glob(f'../test_data/{year}/{month}/*{type_appareil[1]}.Asc')]

C6C20_data = pd.DataFrame()
for file in file_directories:
    asc_data = pd.read_table(file)
    asc_data['Sampling date'] = pd.to_datetime(asc_data['Sampling date']).dt.round('15min')
    asc_data.set_index('Sampling date', inplace=True)
    if "CAL60" in str(file):
        asc_data = asc_data.shift(periods=-1, freq='15min')
    C6C20_data = pd.concat([C6C20_data, asc_data])

C6C20_data.sort_index(inplace=True)

C6C20_data = fill_quarts(
    data=C6C20_data,
    type_appareil=type_appareil[1]
    )
C6C20_data = C6C20_data[~C6C20_data.index.duplicated(keep='first')]

In [218]:
import datetime as dt
import calendar

def filter_month(
        data: pd.DataFrame,
        year: int,
        month: int,
        day: int = 1,
):
    start_date = dt.datetime(year, month, day)
    day_before_start_date = start_date - dt.timedelta(days=1)

    last_day_of_month = calendar.monthrange(year, month)[1]
    end_date = dt.datetime(
        year,
        month,
        last_day_of_month
        )
    data = data[day_before_start_date:end_date]
    return (data)

C2C6_data = filter_month(data=C2C6_data, year=2022, month=2)
C6C20_data = filter_month(data=C6C20_data, year=2022, month=2)


In [221]:
xair_data = pd.concat([C2C6_data, C6C20_data], axis=1,).sort_index()
xair_data.to_csv('../output/test.csv')

In [227]:
xair_data['BENZENE']

,BENZENE,BENZENE
Sampling date,,
2022-01-31 00:15:00,NaN,0.0
2022-01-31 00:30:00,NaN,0.0
2022-01-31 00:45:00,NaN,0.0
2022-01-31 01:00:00,NaN,0.0
2022-01-31 23:30:00,0.00,NaN
...,...,...
2022-02-27 23:00:00,0.03,0.0
2022-02-27 23:15:00,0.03,0.0
2022-02-27 23:30:00,0.17,0.0


In [174]:
dum = pd.DataFrame(xair_data.columns.unique())

In [183]:
dum = dum[1:]
dum['facteur'] = 1

In [199]:
dum =dum.rename(columns={0:"Poll"})
dum.to_csv("../conversion_factors.csv", index=False)

In [12]:
import os

months = [int(x) for x in os.listdir("../test_data")]

In [19]:
if os.path.isdir("../test_data") is not True:
    print("break") 

break
